In [1]:
import torch

In [2]:
num_entries = 5000
db = torch.rand(num_entries) >0.5
db

tensor([0, 0, 1,  ..., 1, 1, 0], dtype=torch.uint8)

In [3]:
db.shape

torch.Size([5000])

In [4]:
#remove entry
db[0:5]
remove_index=2
# db[0:2]db[3:]

torch.cat((db[0:2],db[3:]))

def get_parallel_db(db,remove_index):
    return  torch.cat((db[0:remove_index],db[remove_index+1:]))


In [5]:
get_parallel_db(db,0)

tensor([0, 1, 1,  ..., 1, 1, 0], dtype=torch.uint8)

In [6]:
def get_parallel_dbs(db):
    parallel_dbs = list()
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs 

In [7]:
def create_db_and_parallels(num_entries):

    db = torch.rand(num_entries) >0.5
    pdbs = get_parallel_dbs(db)
    return db,pdbs

In [8]:
db,pdbs = create_db_and_parallels(20)

# lesson 3 

In [9]:
db,pdbs = create_db_and_parallels(20)

In [10]:
def query(db):
    return db.sum()

In [11]:
full_db_result = query(db)

sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result- full_db_result)
    if(db_distance >sensitivity):
        sensitivity = db_distance

In [12]:
sensitivity 

tensor(1)

## sensitivity = L1 sensitivity

# Project 3

In [13]:
def sensitivity(query,n_entries=1000):
    
    db,pdbs = create_db_and_parallels(n_entries)
    full_db_result = query(db)
    sensiti = 0
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result- full_db_result)
        if(db_distance >sensiti):
            sensiti = db_distance
    return sensiti

In [14]:

def query(db):
    return db.float().mean()

In [15]:
sensitivity(query)

tensor(0.0005)

In [16]:
#0.5/5000

# Project: Calculate L1 Sensitivity For Threshold 

In [17]:
def query(db,thresh=5):
    return (db.sum()> thresh).float()

In [18]:
db,pdbs = create_db_and_parallels(10)
query(db)

tensor(0.)

# Perform a Differencing Attack on row 10

In [19]:
db , _ = create_db_and_parallels(100)

In [20]:
pbd = get_parallel_db(db,remove_index=5)

In [21]:
db[5]

tensor(0, dtype=torch.uint8)

In [22]:
# differencing attack using sun query
sum(db) - sum(pbd)

tensor(0, dtype=torch.uint8)

In [23]:
sum(db)

tensor(52, dtype=torch.uint8)

In [24]:
# differencing attack using mean
(sum(db).float()/len(db)) - (sum(pbd).float()/len(pbd))

tensor(-0.0053)

In [25]:
# differencing attack using threshold
(sum(db).float()>49) - (sum(pbd).float()>49)

tensor(0, dtype=torch.uint8)

In [26]:
import numpy as np

# Project Implement Local Differential Privacy
## Randomize Response


In [27]:
db , pdbs = create_db_and_parallels(100)

In [28]:
t_result = torch.mean(db.float())
t_result

tensor(0.4400)

In [29]:
st_c_flip = (torch.rand(len(db))>0.5).float() # 50% hornest 
nd_c_flip = (torch.rand(len(db))>0.5).float() # mean 0.5

In [30]:
st_c_flip

tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
        1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
        1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
        1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 1., 1., 0., 0.])

In [31]:
nd_c_flip

tensor([1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
        0., 1., 1., 0., 0., 1., 0., 1., 0., 0.])

In [32]:
# if first coin flip is 1 then use it 

In [33]:
db.float() * st_c_flip

tensor([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [34]:
# lie

In [35]:
augmented_db=db.float()* st_c_flip+(1-st_c_flip) * (nd_c_flip)  # skew for noice

In [36]:
torch.mean(db.float())

tensor(0.4400)

In [37]:
torch.mean(augmented_db.float())

tensor(0.4400)

In [38]:
# if we knew  
torch.mean(augmented_db.float()) * 2 -0.5

tensor(0.3800)

In [39]:
db_result =torch.mean(augmented_db.float()) * 2 -0.5

In [40]:
# so  Implement query
def query2(db):
    t_result = torch.mean(db.float())
    st_c_flip = (torch.rand(len(db))>0.5).float() # 50% hornest 
    nd_c_flip = (torch.rand(len(db))>0.5).float() # mean 0.5
    augmented_db=db.float()* st_c_flip+(1-st_c_flip) * (nd_c_flip)  # skew for noice
    db_result =torch.mean(augmented_db.float()) * 2 -0.5
    return db_result, t_result

In [41]:
db , pdbs = create_db_and_parallels(100)
private , trueN=query2(db)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

with noise tensor(0.6800)
with out noise tensor(0.5200)


In [42]:
db , pdbs = create_db_and_parallels(1000)
private , trueN=query2(db)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

with noise tensor(0.5160)
with out noise tensor(0.5090)


In [43]:
db , pdbs = create_db_and_parallels(10000)
private , trueN=query2(db)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

with noise tensor(0.4918)
with out noise tensor(0.5022)


# >> LOCAL GURANTEED protect without bias but data hungry for more acc

# Re skew mean for global

In [44]:
def query3(db,noise = 0.2):
    t_result = torch.mean(db.float())
    st_c_flip = (torch.rand(len(db))>noise).float() # 50% hornest 
    
    nd_c_flip = (torch.rand(len(db))>0.5).float() # mean 0.5 Depend on 1st coin flip  
    augmented_db=db.float()* st_c_flip+(1-st_c_flip) * (nd_c_flip)  # skew for noice
    skew_result = augmented_db.float().mean()
    
    private_result = ((skew_result/noise)-0.5) * noise/(1-noise)
    return private_result, t_result

In [45]:
true_dist_mean = 0.7 # 70% say YES
noise_dist_mean = 0.5 # 50/50
augmented_db = db.float()(true_dist_mean + noise_dist_mean) /2

TypeError: 'Tensor' object is not callable

In [ ]:
# 
#revese to adjest a true mean 
# 

In [ ]:
db , pdbs = create_db_and_parallels(100)
private , trueN=query3(db,noise=0.2)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

In [ ]:
db , pdbs = create_db_and_parallels(100)
private , trueN=query3(db,noise=0.5)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

In [ ]:
db , pdbs = create_db_and_parallels(100)
private , trueN=query3(db,noise=0.8)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

In [ ]:
db , pdbs = create_db_and_parallels(1000)
private , trueN=query3(db,noise=0.8)

print("with noise "+str(private) )
print("with out noise "+str(trueN) )

# MORE Noise = more secure 
## - filter unique , understand the general characteristics (consistant)
## - more data => better and secure

# Differential Privacy Query

In [ ]:
db , pdbs = create_db_and_parallels(100)

def M(db):
    np.raplace
    query(db)+ noice
    
    

In [ ]:
epsilon = 0.00001

In [ ]:
import numpy as np

In [ ]:
db , pdbs = create_db_and_parallels(100)

In [ ]:
sum(db) # sensitivity = 0 (0,1)

In [ ]:
def sum_query(db):
    return db.sum()

In [ ]:
def laplacian_machanism(db,query,sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0,beta,1))  # mean 0 to 1 
    return query(db) + noise
    

In [ ]:
def mean_query(db):
    return torch.mean(db.float())

In [ ]:
laplacian_machanism(db,sum_query,1)

In [ ]:
laplacian_machanism(db,mean_query,1/100)# 100 is entry

In [ ]:
#  epsilon too low (highest Protection) but more randomize 

# Generating Differentially Private Labels For a Dataset

In [ ]:
import numpy as np

In [ ]:
num_teachers = 10 # 10 hospital
num_example = 10000# size of dataset
num_label = 10 

In [ ]:
preds = (np.random.rand(num_teachers,num_example)* num_label) .astype(int) #fake label

In [ ]:
preds[:,0] #1st guess of 10 teacher

# Remote Execute

In [46]:
bob.clear_object()

NameError: name 'bob' is not defined